In [ ]:
# Important libraries to install
# pip install langchain
# pip install openai
# pip install -U langchain-community

In [1]:
pip install pymysql

   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os 
os.environ["OPENAI_API_KEY"] = "sk-xxxxxx"
import openai
openai.api_key = "sk-xxxxxxxx"

In [3]:
pip install -U langchain-community

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/2.1 MB 5.0 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/2.1 MB 6.6 MB/s eta 0:00:01
   -------------------------- ------------- 1.4/2.1 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 10.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from langchain.agents import *
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor

In [10]:
db_user = "root"
db_password = "12345"
db_host = "127.0.0.1"
db_name = "student_grade"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [11]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")

In [12]:
toolkit = SQLDatabaseToolkit(db=db,llm=llm)

In [13]:
agen_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [14]:
agen_executor.run("How many tables do we have ?")



> Entering new SQL Agent Executor chain...
I should first find out how many tables are in the database before proceeding with any other queries.
Action: sql_db_list_tables
Action Input: assingments, attendance, names, overallgradeI now know the total number of tables in the database.
Final Answer: 4 tables

> Finished chain.


'4 tables'

In [16]:
agen_executor.run("give the names of all table ")



> Entering new SQL Agent Executor chain...
I should first check the list of tables in the database to see what tables are available.
Action: sql_db_list_tables
Action Input: assingments, attendance, names, overallgradeNow that I know the available tables, I can proceed to give their names.
Action: sql_db_list_tables
Action Input: assingments, attendance, names, overallgradeI should now provide the names of the tables as requested.
Final Answer: assignments, attendance, names, overallgrade

> Finished chain.


'assignments, attendance, names, overallgrade'

In [18]:
agen_executor.run("join table assignments and names as inner join and give results. select only 5 rows ")



> Entering new SQL Agent Executor chain...
I need to first check the tables available in the database before writing the query.
Action: sql_db_list_tables
Action Input: assingments, attendance, names, overallgradeI need to check the schema of the 'assignments' and 'names' tables to see what fields are available for joining.
Action: sql_db_schema
Action Input: assignments, namesError: table_names {'assignments'} not found in databaseI made a mistake in the table name. I need to query the correct table name.
Action: sql_db_schema
Action Input: assignments, namesError: table_names {'assignments'} not found in databaseI need to double-check the table names I entered in the schema query.
Action: sql_db_schema
Action Input: assignments, namesError: table_names {'assignments'} not found in databaseI need to verify the correct table names using the list of tables tool.
Action: sql_db_list_tables
Action Input: assingments, attendance, names, overallgradeI need to correct the typo in the table

'SELECT * \nFROM assignments \nINNER JOIN names \nON assignments.student_id = names.student_id \nLIMIT 5;'

In [24]:
agen_executor.run("give average finalmark from overallgrade table")



> Entering new SQL Agent Executor chain...
I need to calculate the average of the finalmark column in the overallgrade table.
Action: sql_db_query_checker
Action Input: SELECT AVG(finalmark) FROM overallgradeSELECT AVG(finalmark) FROM overallgrade;The query looks correct, I can now execute it to get the average finalmark.
Action: sql_db_query
Action Input: SELECT AVG(finalmark) FROM overallgrade[(47.089700015679,)]The average finalmark from the overallgrade table is 47.09.
Final Answer: 47.09

> Finished chain.


'47.09'